In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import time
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2
np.random.seed(1)

In [2]:
def make_ndarray(shape,value):
    ndarray = []
    try:
        rows = shape[0]
        cols = shape[1]
        chan = shape[2]
        for c in range(chan):
            ndarray.append([])
            for r in range(rows):
                ndarray[c].append([])
                for col in range(cols):
                    ndarray[c][r].append(value)
    except:
        rows = shape[0]
        cols = shape[1]
        for r in range(rows):
            ndarray.append([])
            for col in range(cols):
                ndarray[r].append(value)

    return np.array(ndarray) 

In [3]:
def np_sum(array):
    sum=0
    try:
        rows = array.shape[0]
        cols = array.shape[1]
        chan = array.shape[2]
        for c in range(chan):
            for r in range(rows):
                for col in range(cols):
                    sum+=array[c][r][col]
    except:
        rows = array.shape[0]
        cols = array.shape[1]
        for r in range(rows):
            for col in range(cols):
                sum+=array[r][col]        
    return sum

In [4]:
def np_multiply(array_1,array_2):
    mul_array = make_ndarray(array_1.shape,1.)
    try:
        rows = mul_array.shape[0]
        cols = mul_array.shape[1]
        chan = mul_array.shape[2]
        for c in range(chan):
            for r in range(rows):
                for col in range(cols):
                    mul_array[c][r][col] = array_1[c][r][col]*array_2[c][r][col]
    except:
        rows = mul_array.shape[0]
        cols = mul_array.shape[1]
        for r in range(rows):
            for col in range(cols):
                mul_array[r][col] = array_1[r][col]*array_2[r][col]

    return np.array(mul_array)

In [5]:
def sum_of_arrays(array_1,array_2):
    summed_array = make_ndarray(array_1.shape,0.)
    try:
        rows = summed_array.shape[0]
        cols = summed_array.shape[1]
        chan = summed_array.shape[2]
        for c in range(chan):
            for r in range(rows):
                for col in range(cols):
                    summed_array[c][r][col] = array_1[c][r][col]+array_2[c][r][col]
    except:
        rows = summed_array.shape[0]
        cols = summed_array.shape[1]
        for r in range(rows):
            for col in range(cols):
                summed_array[r][col] = array_1[r][col]+array_2[r][col]

    return np.array(summed_array)

In [6]:
def relu_exp(a):
    for dim1 in range(a.shape[0]):
        for dim2 in range(a.shape[1]):
            for dim3 in range(a.shape[2]):
                for dim4 in range(a.shape[3]):
                    if(a[dim1][dim2][dim3][dim4]<0):
                        a[dim1][dim2][dim3][dim4] = 0
    return a

In [7]:
import cv2
img = np.resize(cv2.imread('2.jpg')/255.,(400,288,3))
print(img.shape)
img = np.resize(img,(1,400,288,3))
img.shape

(400, 288, 3)


(1, 400, 288, 3)

In [8]:
from math import floor,ceil
def zero_pad_same(m,X,strides,n_c,filter_size):
  p_row = ceil(((X.shape[1] - 1) * strides[0] + filter_size[0] - X.shape[1])/2)
  p_col = ceil(((X.shape[2] - 1) * strides[1] + filter_size[1] - X.shape[2])/2)
  #print(p_row,p_col)
  row_num = X.shape[1] + 2 * p_row
  col_num = X.shape[2] + 2 * p_col
  X_padded = np.zeros(shape=(m,row_num, col_num,n_c))
  X_padded[0:m,p_row:p_row+X.shape[1], p_col:p_col+X.shape[2],0:n_c] = X
  return X_padded

In [9]:
def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """

    ### START CODE HERE ### (≈ 2 lines of code)
    # Element-wise product between a_slice and W. Do not add the bias yet.
    s = np.multiply(a_slice_prev,W)
    # Sum over all entries of the volume s.
    Z = np.sum(s)
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    Z = Z + b.astype(float)
    ### END CODE HERE ###
    return Z

In [10]:
def conv_forward(A_prev, W, b,strides,filter_size):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from A_prev's shape (≈1 line)  
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape (≈1 line)
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters" (≈2 lines)
    
    # Initialize the output volume Z with zeros. (≈1 line)
    Z = np.zeros([m, n_H_prev, n_W_prev, n_C])
    #print(Z.shape)
    # Create A_prev_pad by padding A_prev
    A_prev_pad = zero_pad_same(m, A_prev, strides, n_C_prev,filter_size)
    #print(A_prev_pad.shape)

    for i in range(m):                               # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i,:,:,:]                              # Select ith training example's padded activation
        for h in range(n_H_prev):                           # loop over vertical axis of the output volume
            for w in range(n_W_prev):                       # loop over horizontal axis of the output volume
                for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h*strides[0]
                    vert_end = h*strides[0] + f
                    horiz_start = w*strides[1] 
                    horiz_end = w*strides[1] + f
                    
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    
                    
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈1 line)
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[:,:,:,c])
                                        
    ### END CODE HERE ###
    
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H_prev, n_W_prev, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, strides, filter_size)
    
    return Z, cache

In [11]:
def pool_forward(A_prev, hparameters):
    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    """
    
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    #print(n_H,n_W,n_C)
    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))              
    
    ### START CODE HERE ###
    for i in range(m):                         # loop over the training examples
        for h in range(n_H):                     # loop on the vertical axis of the output volume
            for w in range(n_W):                 # loop on the horizontal axis of the output volume
                for c in range (n_C):            # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h*stride
                    vert_end = h*stride +f
                    horiz_start = w*stride
                    horiz_end = w*stride + f
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end,c]
                    
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                    """max_elem = 0
                    (_,dim1,dim2,_) = a_prev_slice.shape
                    for d1 in range(dim1):
                      for d2 in range(dim2):
                        if(a_prev_slice[d1][d2]>max_elem):
                          max_elem = a_prev_slice[d1][d2] """
                    A[i, h, w, c] = np.max(a_prev_slice)
                    
    ### END CODE HERE ###
    
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    
    # Making sure your output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

In [12]:
from math import floor, ceil
def Conv2DTranspose(X, W, padding="valid", strides=(1, 1)):
    # Define output shape before padding
    row_num = (X.shape[0] - 1) * strides[0] + W.shape[0]
    col_num = (X.shape[1] - 1) * strides[1] + W.shape[1]
    output = np.zeros([row_num, col_num])
    # Calculate the output
    for i in range(0, X.shape[0]):
        i_prime = i * strides[0] # Index in output
        for j in range(0, X.shape[1]):
            j_prime = j * strides[1]
            # Insert values
            for k_row in range(W.shape[0]):
                for k_col in range(W.shape[1]):
                    output[i_prime+k_row, j_prime+k_col] += W[k_row, k_col] * X[i, j]
    # Define length of padding
    if padding == "same":
        # returns the output with the shape of (input shape)*(stride)
        p_left = floor((W.shape[0] - strides[0])/2)
        p_right = W.shape[0] - strides[0] - p_left
        p_top = floor((W.shape[1] - strides[1])/2)
        p_bottom = W.shape[1] - strides[1] - p_left
    elif padding == "valid":
        # returns the output without any padding
        p_left = 0
        p_right = 0
        p_top = 0
        p_bottom = 0
    # Add padding
    output_padded = output[p_left:output.shape[0]-p_right, p_top:output.shape[0]-p_bottom]
    return(np.array(output_padded))

In [13]:
def Conv2DTranspose_layer(X, W, padding="same", strides=(1,1)):
    output = []
    (m,height,width,n_c_prev) = X.shape
    (f,f,n_c_prev,filters) = W.shape
    for i in range(m):
        final_output = []
        for f in range(filters):
            base_array = np.zeros((height*strides[0],width*strides[0]))  #initializing with zero
            final_output.append([])
            for c in range(n_c_prev):
                one_channel_output = Conv2DTranspose(X[i,:,:,c],W[:,:,c,f],padding='same',strides=(2,2)) 
                #print(np.array(one_channel_output).shape)
                one_kernel_output = sum_of_arrays(base_array,one_channel_output)
                #print(np.array(one_kernel_output).shape)
                base_array = one_kernel_output
            final_output[-1] = one_kernel_output
        output.append(final_output)
    output = np.array(output)
    return np.moveaxis(output,1,-1)

In [14]:
def concat(X,X_prev):
  (m,height,width,c) = X.shape
  output = []
  for i in range(m):
    height_list = []
    for h in range(height):
      width_list = []
      for w in range(width):
        X1 = list(X[i][h][w])
        X2 = list(X_prev[i][h][w])
        X1.extend(X2)
        width_list.append(X1)
      height_list.append(width_list)
    output.append(height_list)
  return np.array(output)

In [15]:
import cv2
img = np.resize(cv2.imread('2.jpg')/255.,(400,288,3))
print(img.shape)
img = np.resize(img,(1,400,288,3))
img.shape

(400, 288, 3)


(1, 400, 288, 3)

In [16]:
#weight initilization of every layer
init_start = time.time()
W_11 = np.random.randn(3,3,3,32)
b_11 = np.random.randn(1,1,1,32)
W_12 = np.random.randn(3,3,32,32)
b_12 = np.random.randn(1,1,1,32)
W_21 = np.random.randn(3,3,32,64)
b_21 = np.random.randn(1,1,1,64)
W_22 = np.random.randn(3,3,64,64)
b_22 = np.random.randn(1,1,1,64)
W_31 = np.random.randn(3,3,64,128)
b_31 = np.random.randn(1,1,1,128)   
W_32 = np.random.randn(3,3,128,128)
b_32 = np.random.randn(1,1,1,128)
W_41 = np.random.randn(3,3,128,256)
b_41 = np.random.randn(1,1,1,256)
W_42 = np.random.randn(3,3,256,256)
b_42 = np.random.randn(1,1,1,256)
W_51 = np.random.randn(3,3,256,512)
b_51 = np.random.randn(1,1,1,512)
W_52 = np.random.randn(3,3,512,512)
b_52 = np.random.randn(1,1,1,512)
W_61 = np.random.randn(2,2,512,256)
b_61 = np.random.randn(1,1,1,256)
W_62 = np.random.randn(3,3,512,256)
b_62 = np.random.randn(1,1,1,256)
W_63 = np.random.randn(3,3,256,256)
b_63 = np.random.randn(1,1,1,256)
W_71 = np.random.randn(2,2,256,128)
b_71 = np.random.randn(1,1,1,128)
W_72 = np.random.randn(3,3,256,128)
b_72 = np.random.randn(1,1,1,128)
W_73 = np.random.randn(3,3,128,128)
b_73 = np.random.randn(1,1,1,128)
W_81 = np.random.randn(2,2,128,64)
b_81 = np.random.randn(1,1,1,64)
W_82 = np.random.randn(3,3,128,64)
b_82 = np.random.randn(1,1,1,64)
W_83 = np.random.randn(3,3,64,64)
b_83 = np.random.randn(1,1,1,64)
W_91 = np.random.randn(2,2,64,32)
b_91 = np.random.randn(1,1,1,32)
W_92 = np.random.randn(3,3,64,32)
b_92 = np.random.randn(1,1,1,32)
W_93 = np.random.randn(3,3,32,32)
b_93 = np.random.randn(1,1,1,32)
W_out = np.random.randn(1,1,32,1)
b_out = np.random.randn(1,1,1,1)
init_end = time.time()
print('time for weight initilization: ',end='\t')
print(init_end - init_start)

time for weight initilization: 	0.14403247833251953


In [17]:
import pandas as pd
time_data = pd.DataFrame(columns=['image count','executing block','execution time'])
time_data.loc[len(time_data)] = [1,'weight init',0.14792394638061523]
time_data

,image count,executing block,execution time
0,1,weight init,0.147924


In [18]:
np.random.seed(1)
def forward_prop(img,time_data,hparameters = {"stride" : 2, "f": 2}):
    no_img = img.shape[0]
    #conv block 1 <down>
    start = time.time()
    Z_1, cache_conv11 = conv_forward(img, W_11, b_11, strides=(1,1),filter_size=(3,3))
    Z_1, cache_conv12 = conv_forward(Z_1, W_12, b_12, strides=(1,1),filter_size=(3,3))
    A_1, cache_1 = pool_forward(Z_1, hparameters)
    b1 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV1',b1-start]
    print('====== 1 =====')
    #conv block 2 <down>
    Z_2, cache_conv21 = conv_forward(A_1, W_21, b_21, strides=(1,1),filter_size=(3,3))
    Z_2, cache_conv22 = conv_forward(Z_2, W_22, b_22, strides=(1,1),filter_size=(3,3))
    A_2, cache_2 = pool_forward(Z_2, hparameters)
    b2 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV2',b2-b1]
    print('====== 2 =====')
    #conv block 3 <down>
    Z_3, cache_conv31 = conv_forward(A_2, W_31, b_31, strides=(1,1),filter_size=(3,3))
    Z_3, cache_conv32 = conv_forward(Z_3, W_32, b_32, strides=(1,1),filter_size=(3,3))
    A_3, cache_3 = pool_forward(Z_3, hparameters)
    b3 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV3',b3-b2]
    print('====== 3 =====')
    #conv block 4 <down>
    Z_4, cache_conv41 = conv_forward(A_3, W_41, b_41, strides=(1,1),filter_size=(3,3))
    Z_4, cache_conv42 = conv_forward(Z_4, W_42, b_42, strides=(1,1),filter_size=(3,3))
    A_4, cache_4 = pool_forward(Z_4, hparameters)
    b4 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV4',b4-b3]
    print('====== 4 =====')
    #conv block 5 (center)
    Z_5, cache_conv51 = conv_forward(A_4, W_51, b_51, strides=(1,1),filter_size=(3,3))
    Z_5, cache_conv52 = conv_forward(Z_5, W_52, b_52, strides=(1,1),filter_size=(3,3))
    b5 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV5',b5-b4]
    print('====== 5 =====')
    #conv block 6 <up>
    Z_6T = Conv2DTranspose_layer(Z_5, W_61, padding="same", strides=(2,2))
    Z_61 = concat(Z_6T,Z_4)
    Z_6, cache_conv62 = conv_forward(Z_61, W_62, b_62, strides=(1,1),filter_size=(3,3))
    Z_6, cache_conv63 = conv_forward(Z_6, W_63, b_63, strides=(1,1),filter_size=(3,3))
    b6 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV6',b6-b5]
    print('====== 6 =====')
    #conv block 7 <up>
    Z_7T = Conv2DTranspose_layer(Z_6, W_71, padding="same", strides=(2,2))
    Z_71 = concat(Z_7T,Z_3)
    Z_7, cache_conv72 = conv_forward(Z_71, W_72, b_72, strides=(1,1),filter_size=(3,3))
    Z_7, cache_conv73 = conv_forward(Z_7, W_73, b_73, strides=(1,1),filter_size=(3,3))
    b7 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV7',b7-b6]
    print('====== 7 =====')
    #conv block 8 <up>
    Z_8T = Conv2DTranspose_layer(Z_7, W_81, padding="same", strides=(2,2))
    Z_81 = concat(Z_8T,Z_2)
    Z_8, cache_conv82 = conv_forward(Z_81, W_82, b_82, strides=(1,1),filter_size=(3,3))
    Z_8, cache_conv83 = conv_forward(Z_8, W_83, b_83, strides=(1,1),filter_size=(3,3))
    b8 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV8',b8-b7]
    print('====== 8 =====')
    #conv block 9 <up>
    Z_9T = Conv2DTranspose_layer(Z_8, W_91, padding="same", strides=(2,2))
    Z_91 = concat(Z_9T,Z_1)
    Z_9, cache_conv92 = conv_forward(Z_91, W_92, b_92, strides=(1,1),filter_size=(3,3))
    Z_9, cache_conv93 = conv_forward(Z_9, W_93, b_93, strides=(1,1),filter_size=(3,3))
    b9 = time.time()
    time_data.loc[len(time_data)] = [no_img,'CONV9',b9-b8]
    print('====== 9 =====')
    #conv block 10 <output>
    Z_out, cache_conv_out = conv_forward(Z_9, W_out, b_out, strides=(1,1),filter_size=(1,1))
    b10 = time.time()
    print('====== 10 =====')
    time_data.loc[len(time_data)] = [no_img,'CONV10',b10-b9]
    total_time = b10-start
    time_data.loc[len(time_data)] = [no_img,'Total',total_time]
    return Z_out


In [ ]:
output = forward_prop(img,time_data,hparameters = {"stride" : 2, "f": 2})
output.shape

====== 1 =====
====== 2 =====
====== 3 =====
====== 4 =====
====== 5 =====
====== 6 =====
